In [25]:
import os
import copy
import pathlib
import pandas as pd
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning.foundation_models.extensions.langchain import (
    WatsonxLLM,
)
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.schema.embeddings import Embeddings
from typing import List
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()


class MiniLML6V2EmbeddingFunctionLangchain(Embeddings):
    MODEL = SentenceTransformer("all-MiniLM-L6-v2")

    def embed_documents(self, texts):
        return MiniLML6V2EmbeddingFunctionLangchain.MODEL.encode(texts).tolist()

    def embed_query(self, query):
        return MiniLML6V2EmbeddingFunctionLangchain.MODEL.encode([query]).tolist()[0]


class MultilingualMiniLML6V2EmbeddingFunctionLangchain(Embeddings):
    MODEL = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

    def embed_documents(self, texts):
        return MiniLML6V2EmbeddingFunctionLangchain.MODEL.encode(texts).tolist()

    def embed_query(self, query):
        return MiniLML6V2EmbeddingFunctionLangchain.MODEL.encode([query]).tolist()[0]

In [26]:
def translate(text, llm, direction):
    if direction=="ms2en":
        prompt = f"""
        Translate the following from Malay to English, but avoid translating the words "bumiputera", "Bumiputera" and "bumi putera":
    
        {text}
        .
        """
    else:
        prompt = f"""
        Translate the following from English to Malay:
    
        {text}
        .
        """
        

    return(llm(prompt))

In [27]:
#load data

import pandas as pd
test_q_eng = pd.read_excel("../data/test_questions.xlsx", sheet_name="English")
test_q_ms = pd.read_excel("../data/test_questions.xlsx", sheet_name="Malay")

In [28]:
import fitz
import camelot
from langchain.docstore.document import Document

class Conversation:
    def __init__(self, english_source=True, llama=True, hook_translation=False):
        self.hook_translation = hook_translation
        self.translation_llm = WatsonxLLM(Model(
            model_id=ModelTypes.MT0_XXL,
            credentials={
                "apikey": os.getenv("IBM_API_KEY"),
                "url": "https://us-south.ml.cloud.ibm.com",
            },
            params={
                GenParams.MAX_NEW_TOKENS: 1024,
                GenParams.TEMPERATURE: 0,
                GenParams.RANDOM_SEED: 12345,
            },
            project_id=os.getenv("PROJECT_ID"),))
        if english_source:
            self.pdf = self.read_pdf("../data/BUKU GARIS PANDUAN 2022 - 03.11.2023 (edit klasifikasi mekanikal - adm1 (EN).pdf")
            self.embeddings = MiniLML6V2EmbeddingFunctionLangchain()
        else:
            self.pdf = self.read_pdf("../data/BUKU GARIS PANDUAN 2022 - 03.11.2023 (edit klasifikasi mekanikal - adm1.pdf")
            self.embeddings = MultilingualMiniLML6V2EmbeddingFunctionLangchain()
        self.text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)
        self.K_DOCS = 5
        self.llama = llama


        if llama:
            model = Model(
            model_id=ModelTypes.LLAMA_2_70B_CHAT,
            credentials={
                "apikey": os.getenv("IBM_API_KEY"),
                "url": "https://us-south.ml.cloud.ibm.com",
            },
            params={
                GenParams.MAX_NEW_TOKENS: 1024,
                GenParams.TEMPERATURE: 0,
                GenParams.RANDOM_SEED: 12345,
            },
            project_id=os.getenv("PROJECT_ID"),)
        else:
            model = Model(
            model_id=ModelTypes.MT0_XXL,
            credentials={
                "apikey": os.getenv("IBM_API_KEY"),
                "url": "https://us-south.ml.cloud.ibm.com",
            },
            params={
                GenParams.MAX_NEW_TOKENS: 1024,
                GenParams.TEMPERATURE: 0,
                GenParams.RANDOM_SEED: 12345,
            },
            project_id=os.getenv("PROJECT_ID"),)
            

        self.llm = WatsonxLLM(model=model)

        self.QUESTION_TEMPLATE = """
        Context information is below.
        ---------------------
        {{context}}
        ---------------------
        
        Query: {{question}}
        
        Given the context information and not prior knowledge respond to the query in a brief and concise manner. If the question is asked in Malay, answer in Malay. If the question is asked in English, answer in English.
        Avoid statements like 'Based on the context, ...' or 'According to the provided context ...' or anything along those lines.
        
        Based on the context, if the meaning/intent of the query is not clear, respond by asking for additional information that will enable you to determine the intent. Then, end the response. 
        If the intent behind the query is clear, but you don't know the answer to a query, say "I do not know".
        
        Remember: Respond to the query in a brief and concise manner, and avoid talking about yourself. If the question is asked in Malay, answer in Malay. If the question is asked in English, answer in English.

        Let us think step by step.
                
        Response: 
        
        """
        
        
        self.TELL_ME_MORE_TEMPLATE = """
        Context information is below.
        ---------------------
        {{context}}
        ---------------------
        
        Query: {{question}}
        
        Given the context information and not prior knowledge respond to the query in a more detailed manner. If the question is asked in Malay, answer in Malay. If the question is asked in English, answer in English.
        
        Do not repeat the previous answer.
        
        Avoid statements like 'Based on the context, ...' or 'According to the provided context ...' or anything along those lines.
        
        Based on the context, if the meaning/intent of the query is not clear, respond by asking for additional information that will enable you to determine the intent. Then, end the response. 
        If the intent behind the query is clear, but you don't know the answer to a query, say "I do not know".
        
        Remember: Respond to the query in a more detailed manner, and avoid talking about yourself. If the question is asked in Malay, answer in Malay. If the question is asked in English, answer in English.
        
        Let us think step by step.
        
        Response: 
        
        """

        self.embed_docs(english_source=english_source)

    def embed_docs(self, english_source):
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
        if english_source:
            docs = text_splitter.split_documents(self.pdf)
            db = FAISS.from_documents(docs, self.embeddings)
            db.save_local("../code-engine/db")
            self.db = FAISS.load_local("../code-engine/db", self.embeddings)
        else:
            docs = text_splitter.split_documents(self.pdf)
            db = FAISS.from_documents(docs, self.embeddings)
            db.save_local("../code-engine/db2")
            self.db = FAISS.load_local("../code-engine/db2", self.embeddings)

    def read_pdf(self, path):
        pdf_file = fitz.open(path)
        docs = []
        for page_index in tqdm(range(len(pdf_file))):
            # get the page itself
            page = pdf_file[page_index]
            tables = "\n\n\n".join([i.to_pandas().to_html() for i in page.find_tables(horizontal_strategy="text").tables])# + f"\n\n\n\nHTML TABLES:\n\n\n\n{tables}"
            docs.append(Document(page_content=page.get_text(), metadata={"filename":page_index}))
            #print(page.get_text())
        self.docs = docs
        return docs
    
    def build_prompt(self,messages):
        if self.llama:
            B_INST, E_INST = "[INST]", "[/INST]"
            B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
            BOS, EOS = "<s>", "</s>"
            DEFAULT_SYSTEM_PROMPT = """
            "You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content."
            """
            if not messages:
                return None
            messages_ = copy.deepcopy(messages)
            messages_ = [x["a"] if "a" in x else x["u"] for x in messages_]
            messages_[0] = "".join([B_SYS, DEFAULT_SYSTEM_PROMPT, E_SYS, messages_[0]])
            messages_list = []
            for i, x in enumerate(messages_):
                if i % 2 == 0:
                    messages_list.append(f"{BOS}{B_INST} {x.strip()} {E_INST}")
                else:
                    messages_list.append(f" {x.strip()} {EOS}")
            prompt = "".join(messages_list)
        else:
            if not messages:
                return None
            messages_ = copy.deepcopy(messages)
            messages_ = [x["a"] if "a" in x else x["u"] for x in messages_]
            messages_[0] = "".join([messages_[0]])
            messages_list = []
            for i, x in enumerate(messages_):
                if i % 2 == 0:
                    messages_list.append(f"{x.strip()}")
                else:
                    messages_list.append(f"{x.strip()}")
            prompt = "".join(messages_list)
            

        return prompt


    def chat(self,messages):
        if self.hook_translation:
            messages[-1]["u"] = translate(messages[-1]["u"], llm=self.translation_llm, direction="en2ms")
        question = messages[-1]["u"]
        if "tell me more" in question:
            search_results = self.db.similarity_search(messages[-3]["u"], k=self.K_DOCS)
            context = " ".join([x.page_content for x in search_results])
            messages[-1]["u"] = self.TELL_ME_MORE_TEMPLATE.replace(
                "{{context}}", context
            ).replace("{{question}}", messages[-3]["u"])
        else:
            search_results = self.db.similarity_search(question, k=self.K_DOCS)
            context = " ".join([x.page_content for x in search_results])
            messages[-1]["u"] = self.QUESTION_TEMPLATE.replace("{{context}}", context).replace(
                "{{question}}", question
            )
    
        prompt = self.build_prompt(messages)
        assistant = self.llm(prompt).strip().replace("•", "*").replace("```", "")
        if self.hook_translation:
            assistant = translate(assistant, llm=self.translation_llm, direction="ms2en")
        source = ""
        if "I do not know" not in assistant:
            source = "\n\nSource:\n- " + "\n- ".join(dict.fromkeys([str(x.metadata["filename"]) for x in search_results]))
        messages[-1]["u"] = question
        messages.append({"a": f"{assistant}"}) #{source}
        return messages, context, prompt
                 


In [18]:
#default setting - use translated pdf
chatter = Conversation()
generated_default = []
for question in test_q_eng["Questions"]:
    messages, context, prompt = chatter.chat([{"u": question}])
    a = messages[-1]["a"]
    generated_default.append(a)
    print(f"Q: {question}")
    print(f"G: {a}")
    print()
    


100%|█████████████████████████████████████████| 176/176 [00:04<00:00, 43.36it/s]


Q: If I only have 30% share in my company, can I 
apply for bumiputera status? 
G: Saya tidak tahu.

(I don't know)

In this case, I don't have enough information to give a clear answer to the query. The query asks if a company with 30% share can apply for Bumiputera status, but the context information only provides the requirements for Bumiputera status, not the specific requirements for a company with 30% share. Therefore, I cannot provide a clear answer and will ask for additional information to determine the intent of the query.

Q: What is UPKJ Bumiputera Status Requirement?
G: Syarat-syarat status Bumiputera UPKJ ialah sebagai berikut:

1. Kepemilikan saham individu atau korporat/nama must be at least 51% bumiputera.
2. Nama dan alamat syarikat/koperasi/korporat must be registered with the Companies Commission of Malaysia (SSM).
3. Syarikat/koperasi/korporat must have a minimum paid-up capital of RM50,000.
4. Syarikat/koperasi/korporat must have a minimum of 30% bumiputera employ

In [ ]:
# test malay - only for experimental purpose
chatter = Conversation(english_source=False, llama=True)
generated_ms = []
for question in test_q_ms["Questions"]:
    messages, context, prompt = chatter.chat([{"u": question}])
    a = messages[-1]["a"]
    generated_ms.append(a)
    print(f"Q: {question}")
    print(f"G: {a}")
    print()

100%|█████████████████████████████████████████| 176/176 [00:10<00:00, 16.51it/s]


Q: What is UPKJ Bumiputera Status Requirement?
G: Bumiputera Status Requirement for UPKJ is as follows:

1. The company must be registered with the Companies Commission of Malaysia (SSM) under the Companies Act 1965 or the Companies Act 2016.
2. The company must have a minimum paid-up capital of RM50,000.
3. The company must have a good track record in the construction industry, including experience in carrying out projects for the government or private sector.
4. The company must have a sound financial capability, including a minimum net worth of RM500,000.
5. The company must have a proper accounting system and be able to provide financial statements that are audited by a chartered accountant.
6. The company must have a valid tax registration number and be up to date with its tax payments.
7. The company must have a valid GST registration number, if applicable.
8. The company must have a valid license from the relevant authorities, such as the Ministry of Works, Ministry of Transport

In [7]:
english_results = pd.DataFrame({"question":test_q_eng["Questions"], "generated_default":generated_default})
malay_results = pd.DataFrame({"question":test_q_ms["Questions"], "generated_answer":generated_ms})

In [8]:
pd.set_option('display.max_colwidth', 0)
english_results

,question,generated_default
0,"If I only have 30% share in my company, can I \napply for bumiputera status?","I do not know.\n\nHowever, based on the information provided, it seems that the requirement for Bumiputera status is that the company/firm/cooperative/corporation must be owned at least 51% by Bumiputera Sarawak. Therefore, if you only have 30% share in your company, you may not be eligible for Bumiputera status. But I'm not sure about the specific details of your case, so I can't give a definitive answer. Can you provide more information about your company's ownership structure and the specific requirements for Bumiputera status in your area?"
1,What is UPKJ Bumiputera Status Requirement?,"UPKJ Bumiputera Status requirement is a set of criteria that must be met by a company or individual to qualify for Bumiputera status, including minimum requirements for Bumiputera ownership, management, and control."
2,Can we change our Bumiputera interview Date?,"Sorry, the given context information is not sufficient to answer your question. Can you provide more information or clarify your query?"
3,If yes how do we change our Bumiputera Interview date?,"""To change the Bumiputera interview date, you can do so through the OLAM system, but it's only allowed once before a period of three days from the original interview date, and only if you have a valid reason."""
4,I have fail my bumiputera status and the system does not allow me to apply again?,"Saya tidak tahu. (I don't know)\n\nHowever, I can suggest some possible reasons why the system may not allow you to apply again:\n\n1. The system may have detected that you have previously applied for Bumiputera status and were rejected.\n2. You may have provided incorrect or incomplete information in your previous application.\n3. You may not meet the eligibility criteria for Bumiputera status.\n\nTo resolve this issue, you can try the following:\n\n1. Check your previous application status and see if there were any errors or missing information.\n2. Ensure that you meet the eligibility criteria for Bumiputera status.\n3. Contact the relevant authorities or the system administrator for assistance.\n\nRemember to always provide accurate and complete information when applying for Bumiputera status, and ensure that you meet the eligibility criteria to avoid any issues or rejections."
5,Do I need to apply for bumiputera status if I allready have bumi status for mechanical category?,"Based on the context, it seems that you are asking if you need to apply for bumiputera status if you already have bumi status for mechanical category. However, to confirm, can you please provide more information on your current bumi status and the specific context you are referring to? This will help me better understand your question and provide a more accurate response."
6,"I wish to apply bumi putera status for my consultancy firm, can you help?","""Saya tidak mengetahui"" (I don't know) as I require more information to determine the intent behind your query. Can you please provide more context or clarify your question?"
7,What are the document needed if I wish to apply for bumi putera status?,Documents needed for Bumiputera status application include:\n\n* Company registration certificate\n* Business license\n* EPF registration certificate (if applicable)\n* Bank statement or financial signature confirmation letter/company check that has been certified by the bank\n* Birth certificate of owner/holder of shares/director\n* Signatory condition\n* Name of owner and share handle.
8,How can I renew my company bumiputera status?,"""You can renew your company's bumiputera status by submitting an application through the OLAM system and ensuring that all required documents and information are up to date and accurate."""
9,What is Bumiputera Sarawak?,"Bumiputera Sarawak refers to a status granted to companies or individuals that are owned or controlled by indigenous people of Sarawak, Malaysia, and meet certain criteria

In [9]:
#only for internal testing
malay_results

,question,generated_answer
0,Boleh x ktk terangkan apa proses dalam \nmemohon status bumiputera oh? confuse kmk,"1. Hantar permohonan melalui sistem OLAM 2. Semakan Ya Tidak 3(a). Permohonan dikembalikan kepada pemohon untuk pembetulan 3(b). Temuduga 4. Keputusan Lulus Ya Tidak 5(a). Status Tidak Lulus Di Dalam OLAM TAMAT Nota: Jika tidak lulus, Pemohon boleh memohon semula status bumiputera selepas tempoh enam (6) bulan. 5(b). Cetak Sijil TAMAT"
1,"Mun kmk gagal interview untuk tok, apa jadi?",I do not know
2,"kmk mok order laksa sarawak \nsimanggang, macam nie mok beli oh?",I do not know
3,Apakah senarai semak untuk permohonan \nstatus bumiputera bagi syarikat jenis \nenterprise?,"1. SSM Info Korporat yang terkini* 2. Laporan bank atau surat pengesahan penandatangan kewangan/ cek syarikat yang telah disahkan oleh Bank (dengan maklumat seperti berikut):* a. Nama syarikat/ koperasi/ perbadanan beserta no. pendaftaran b. Nama bank, no. akaun dan tarikh akaun dibuka c. Nama pemilik dan pegangan saham d. Penama yang diberikuasa menantangan cek e. Signatory condition 3. KWSP (Jika berkenaan)** 4. Sijil kelahiran pemilik/ pemegang saham/ pengarah* 5. Gambar a. di luar bangunan pejabat (depan pintu), b. di dalam pejabat, c. di luar bangunan (dari jarak jauh- nampak kawasan pejabat)"
4,Apakah syarat utama untuk mendapatkan pengiktirafan Status Bumiputera?,a. Majoriti atau sekurang-kurangnya 51% saham/ ekuiti adalah dipegang oleh Bumiputera Sarawak dan pemegang saham utama mestilah Bumiputera Sarawak.
5,Apakah syarat mendapatkan Status Bumiputera bagi Syarikat jenis Enterprise?,"2. Laporan bank atau surat pengesahan penandatangan kewangan/ cek syarikat yang telah disahkan oleh Bank (dengan maklumat seperti berikut):* a. Nama syarikat/ koperasi/ perbadanan beserta no. pendaftaran b. Nama bank, no. akaun dan tarikh akaun dibuka c. Nama pemilik dan pegangan saham d. Penama yang diberikuasa menantangan cek e. Signatory condition 3. KWSP (Jika berkenaan)** 4. Sijil kelahiran pemilik/ pemegang saham/ pengarah* 5. Gambar a. di luar bangunan pejabat (depan pintu), b. di dalam pejabat, c. di luar bangunan (dari jarak jauh- nampak kawasan pejabat)"
6,Apakah syarat mendapatkan Status Bumiputera bagi Syarikat Bhd atau Sdn Bhd?,a. Majoriti atau sekurang-kurangnya 51% saham/ ekuiti adalah dipegang oleh Bumiputera Sarawak dan pemegang saham utama mestilah Bumiputera Sarawak.
7,Berapa pecahan saham / ekuiti yang perlu dipegang oleh Bumiputera Sarawak bagi Syarikat Enterprise?,51%
8,Berapa pecahan pemilikan saham / ekuiti secara korporat yang perlu dimiliki oleh Bumiputera Sarawak untuk mendapatkan Status Bumiputera?,51%
9,"Jika permohonan Status Bumiputera gagal, adakah permohonan semula dibenarkan?",Pemohon boleh memohon semula status bumiputera selepas tempoh enam (6) bulan


In [29]:
#look @ table
chatter = Conversation(english_source=False, llama=True)

100%|█████████████████████████████████████████| 176/176 [00:11<00:00, 15.95it/s]


In [32]:
chatter.docs[1]

Document(page_content='(ii) \n \n \n \nPejabat  \nSetiausaha Kewangan Negeri \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nUnit Pendaftaran Kontraktor \nDan Juruperunding (UPKJ) \nTingkat 7, Wisma Satok \n93400 Kuching \n \nTel : 082 - 235710/ 235810 \nFaks : 082 - 2515 \n', metadata={'filename': 1})